In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [2]:
df = pd.read_csv('C:/Users/JenMing/Desktop/MBTI/LSTM/mbti_to_LSTM_DF.csv')
df.head()

,type,posts
0,INFJ,"['INFP', 'INFP', 'INFJ', 'ENFP', 'ISTP', 'INTP..."
1,ENTP,"['INTJ', 'INTP', 'ENFP', 'INTJ', 'INTP', 'INTP..."
2,INTP,"['INTJ', 'INFP', 'INFP', 'INTP', 'INTP', 'INTJ..."
3,INTJ,"['INTJ', 'ISFJ', 'INFP', 'INTP', 'INTP', 'INTP..."
4,ENTJ,"['ENTJ', 'INTP', 'ENFP', 'INTP', 'ENTJ', 'INTJ..."


In [3]:
# 編碼轉換
personality_mapping = {'INFJ': 0,
                        'ENTP': 1,
                        'INTP': 2,
                        'INTJ': 3,
                        'ENTJ': 4,
                        'ENFJ': 5,
                        'INFP': 6,
                        'ENFP': 7,
                        'ISFP': 8,
                        'ISTP': 9,
                        'ISFJ': 10,
                        'ISTJ': 11,
                        'ESTP': 12,
                        'ESFP': 13,
                        'ESTJ': 14,
                        'ESFJ': 15 }

In [4]:
# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    

for index, row in df.iterrows():
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    
    
    dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((dialogue_ids, target_personality_id))

In [5]:
# 動態計算 input_size
max_dialogue_length = max(len(dialogue) for dialogue, _ in encoded_data)
input_size = max_dialogue_length

In [6]:
# 填充序列並轉換為張量
padded_dialogues = [torch.tensor(dialogue, dtype=torch.float32) for dialogue, _ in encoded_data]
padded_dialogues = pad_sequence(padded_dialogues, batch_first=True)

target_personality = torch.tensor([target for _, target in encoded_data], dtype=torch.float32)

In [7]:
# 将 padded_dialogues 从2D转换为1D张量
padded_dialogues_1d = padded_dialogues.view(-1)

# 创建新的目标数据和新的对话数据
new_target_personality = []
new_padded_dialogues = []

# 遍历原始数据，并为每个对话生成对应的目标数据
for i in range(len(encoded_data)):
    target = encoded_data[i][1]  # 原始的目标数据
    dialogues = encoded_data[i][0]  # 原始的对话数据

    # 为每个对话生成新的对话数据
    for dialogue in dialogues:
        new_target_personality.append(target)
        new_padded_dialogues.append([dialogue])

# 转换为张量
new_target_personality = torch.tensor(new_target_personality, dtype=torch.float32)
new_padded_dialogues = pad_sequence([torch.tensor(dialogue, dtype=torch.float32) for dialogue in new_padded_dialogues], batch_first=True)

# 检查结果的形状
print(new_padded_dialogues.shape)
print(new_target_personality.shape)

torch.Size([392847, 1])
torch.Size([392847])


In [8]:
# 定义 ANN 模型
class PersonalityPredictionANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.5):
        super(PersonalityPredictionANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

In [9]:
# 資料集切分為訓練集和驗證集
train_dialogues, val_dialogues, train_target, val_target = train_test_split(new_padded_dialogues, new_target_personality, test_size=0.15, random_state=42)

# 初始化模型
hidden_size = 128
output_size = len(personality_mapping)
input_size = 1
model = PersonalityPredictionANN(input_size, hidden_size, hidden_size, output_size)

In [10]:
# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [11]:
#早停
patience = 10  # 設定早期停止的耐心值
best_val_loss = float('inf')
counter = 0  # 用於計算連續的驗證損失沒有改善的次數

In [12]:
with open("C:/Users/JenMing/Desktop/MBTI/ANN/Model/1d/note.txt", "w") as f:
    num_epochs = 60
    dimension_counts = {'E/I': 0,
                        'S/N': 0,
                        'T/F': 0,
                        'J/P': 0}
    item_count = 0
    for epoch in range(num_epochs):
        model.train()  # 將模型設置為訓練模式
        total_loss = 0.0

        for dialogue_batch, target_batch in zip(train_dialogues, train_target):
            optimizer.zero_grad()

            outputs = model(dialogue_batch)
            loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dialogues)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")
        f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}\n")
        
        # 驗證模型
        model.eval()  # 將模型設置為評估模式
        correct_predictions = 0
        total_samples = len(val_dialogues)
        val_loss = 0.0

        with torch.no_grad():
            for dialogue_batch, target_batch in zip(val_dialogues, val_target):
                outputs = model(dialogue_batch)
                loss = criterion(outputs, target_batch.long())  # 使用交叉熵損失
                val_loss += loss.item()

                predicted_class = torch.argmax(outputs).item()
                true_class = target_batch.item()

                for personality, value in personality_mapping.items():
                    if value == predicted_class:
                        mbti_labels_pre = personality
                        break
                for personality, value in personality_mapping.items():
                    if value == int(true_class):
                        mbti_labels_tru = personality 
                        break

                for n in range(4):
                    if n == 0:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['E/I'] += 1
                    elif n == 1:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['S/N'] += 1
                    elif n == 2:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['T/F'] += 1
                    elif n == 3:
                        if mbti_labels_pre[n] == mbti_labels_tru[n]:
                            dimension_counts['J/P'] += 1

                if predicted_class == true_class:
                    correct_predictions += 1

        item_count += total_samples
        average_val_loss = val_loss / len(val_dialogues)
        accuracy = correct_predictions / total_samples
        print(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%")
        f.write(f"Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy*100:.4f}%\n")
        
        # 檢查驗證損失是否改善
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            counter = 0
        else:
            counter += 1

        # 如果連續一定次數（耐心值）驗證損失沒有改善，則停止訓練
        if counter >= patience:
            print(f"Early Stopping: Validation loss has not improved for {patience} epochs. Stopping training.")
            break

    EI_counts = dimension_counts['E/I']
    SN_counts = dimension_counts['S/N']
    TF_counts = dimension_counts['T/F']
    JP_counts = dimension_counts['J/P']
    print(f'E.I: {EI_counts}/{item_count} ')
    print('Accuracy: '+ str(EI_counts/item_count)+'\n')
    print(f'S.N: {SN_counts}/{item_count} ')
    print('Accuracy: '+ str(SN_counts/item_count)+'\n')
    print(f'T.F: {TF_counts}/{item_count} ')
    print('Accuracy: '+ str(TF_counts/item_count)+'\n')
    print(f'J.P: {JP_counts}/{item_count} ')
    print('Accuracy: '+ str(JP_counts/item_count)+'\n')
    
    f.write(f'E.I: {EI_counts}/{item_count} ')
    f.write('Accuracy: '+ str(EI_counts/item_count)+'\n')
    f.write(f'S.N: {SN_counts}/{item_count} ')
    f.write('Accuracy: '+ str(SN_counts/item_count)+'\n')
    f.write(f'T.F: {TF_counts}/{item_count} ')
    f.write('Accuracy: '+ str(TF_counts/item_count)+'\n')
    f.write(f'J.P: {JP_counts}/{item_count} ')
    f.write('Accuracy: '+ str(JP_counts/item_count)+'\n')

Epoch [1/60], Loss: 2.2855
Validation Loss: 2.2625, Validation Accuracy: 22.5784%
Epoch [2/60], Loss: 2.2722
Validation Loss: 2.2628, Validation Accuracy: 22.0167%
Epoch [3/60], Loss: 2.2678
Validation Loss: 2.2652, Validation Accuracy: 22.0167%
Epoch [4/60], Loss: 2.2679
Validation Loss: 2.2642, Validation Accuracy: 22.0167%
Epoch [5/60], Loss: 2.2690
Validation Loss: 2.2637, Validation Accuracy: 22.0167%
Epoch [6/60], Loss: 2.2670
Validation Loss: 2.2629, Validation Accuracy: 22.0167%
Epoch [7/60], Loss: 2.2664
Validation Loss: 2.2630, Validation Accuracy: 22.0167%
Epoch [8/60], Loss: 2.2659
Validation Loss: 2.2615, Validation Accuracy: 22.4274%
Epoch [9/60], Loss: 2.2651
Validation Loss: 2.2607, Validation Accuracy: 22.6106%
Epoch [10/60], Loss: 2.2644
Validation Loss: 2.2618, Validation Accuracy: 22.4274%
Epoch [11/60], Loss: 2.2623
Validation Loss: 2.2575, Validation Accuracy: 22.5784%
Epoch [12/60], Loss: 2.2594
Validation Loss: 2.2556, Validation Accuracy: 22.6106%
Epoch [13/60]

In [13]:
torch.save(model.state_dict(), "C:/Users/JenMing/Desktop/MBTI/ANN/Model/1d/best_model.pth")